<a href="https://colab.research.google.com/github/uttkarsh123-shiv/CineSentiment/blob/main/CineSentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow
import warnings

warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('./drive/MyDrive/IMDB_Dataset.csv')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
data.shape

(50000, 2)

In [4]:
type(data)

pandas.core.frame.DataFrame

In [5]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [6]:
data["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [7]:
# positive -> 1
# negative -> 0
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

In [8]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [9]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0
49999,No one expects the Star Trek movies to be high...,0


In [10]:
data["sentiment"].value_counts()

,count
sentiment,
1,25000
0,25000


In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [12]:
train_data, test_data = train_test_split(data, test_size = 0.2, random_state=42)

In [13]:
train_data.shape

(40000, 2)

In [14]:
test_data.shape

(10000, 2)

In [15]:
tokenizer = Tokenizer(num_words = 5000)
test = tokenizer.fit_on_texts(train_data["review"])

In [16]:
x_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [17]:
x_train

array([[1935,    1, 1200, ...,  205,  351, 3856],
       [   3, 1651,  595, ...,   89,  103,    9],
       [   0,    0,    0, ...,    2,  710,   62],
       ...,
       [   0,    0,    0, ..., 1641,    2,  603],
       [   0,    0,    0, ...,  245,  103,  125],
       [   0,    0,    0, ...,   70,   73, 2062]], dtype=int32)

In [18]:
x_test

array([[   0,    0,    0, ...,  995,  719,  155],
       [  12,  162,   59, ...,  380,    7,    7],
       [   0,    0,    0, ...,   50, 1088,   96],
       ...,
       [   0,    0,    0, ...,  125,  200, 3241],
       [   0,    0,    0, ..., 1066,    1, 2305],
       [   0,    0,    0, ...,    1,  332,   27]], dtype=int32)

In [19]:
y_train = train_data["sentiment"]
y_test = test_data["sentiment"]

In [20]:
y_train

,sentiment
39087,0
30893,0
45278,1
16398,0
13653,0
...,...
11284,1
44732,1
38158,0
860,1


In [21]:
# LSTM MODEL

In [22]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length = 200))
model.add(LSTM(128, dropout=0.2,recurrent_dropout=0.2 ))
model.add(Dense(1, activation="sigmoid"))

model.summary()


In [23]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [24]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [25]:
model.fit(x_train, y_train, epochs=5, batch_size=64,validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 231s 442ms/step - accuracy: 0.7112 - loss: 0.5354 - val_accuracy: 0.7966 - val_loss: 0.4899
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 213s 426ms/step - accuracy: 0.8077 - loss: 0.4327 - val_accuracy: 0.8449 - val_loss: 0.3664
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 217s 434ms/step - accuracy: 0.8687 - loss: 0.3259 - val_accuracy: 0.8565 - val_loss: 0.3545
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 208s 417ms/step - accuracy: 0.8915 - loss: 0.2760 - val_accuracy: 0.8676 - val_loss: 0.3350
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 217s 434ms/step - accuracy: 0.8935 - loss: 0.2654 - val_accuracy: 0.8694 - val_loss: 0.3275


In [26]:
model.save("model.h5")

In [31]:
import joblib
joblib.dump(tokenizer, "tokenizer.pkl")

['tokenizer.pkl']

In [27]:
loss, accuracy = model.evaluate(x_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 42s 131ms/step - accuracy: 0.8789 - loss: 0.3140


In [28]:
print(loss)

0.314201682806015


In [29]:
print(accuracy)

0.8783000111579895


In [30]:
# Building Predictive System

In [38]:
def predictive_system(review):
  sequences = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequences, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [40]:
predictive_system("This movie was fantastic")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step


'positive'

In [46]:
predictive_system("I did not liked at ending")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step


'negative'